# ALS transect 3D visualization

This notebook illustrates transect visualization using CVL 3D Viz software


## Objectives

Demonstrate visualization of X,Y,Z timeseries together with the Sentinel-1 image as background

## Prerequisites

Software prerequisites
- pip3
- cvl 3d viz library
- requests

System libraries not provided via CVL have been installed using conda/mamba package manager.

Install CVL 3D viz library for steering the visualization in-browser app 


```
# Use preceeding exclamation mark if installing from within the notebook, othewise add URL below to requirements file
!pip3 install git+https://github.com/CryosphereVirtualLab/cvl-3d-viz#egg=cvl-3d-viz
```


In [1]:
!pip3 install git+https://github.com/CryosphereVirtualLab/cvl-3d-viz#egg=cvl-3d-viz

  Cloning https://github.com/CryosphereVirtualLab/cvl-3d-viz to /tmp/pip-install-im071avl/cvl-3d-viz_76067c9ef0cb4bcea23d39dc42c6dcc7
  Running command git clone --filter=blob:none --quiet https://github.com/CryosphereVirtualLab/cvl-3d-viz /tmp/pip-install-im071avl/cvl-3d-viz_76067c9ef0cb4bcea23d39dc42c6dcc7
  Resolved https://github.com/CryosphereVirtualLab/cvl-3d-viz to commit a1b6166e7c2d2b15379c7598ef82ab497e97e4b9
  Preparing metadata (setup.py) ... done


## Starting CVL 3d Viz server
The server is necessary to communicate between the notebook and the browser application. The server needs to be available, configured and running locally.


In [2]:
# Obtain server code
# !if [ ! -d ~/cvl-3d-viz ]; then git clone https://github.com/CryosphereVirtualLab/cvl-3d-viz.git ~/cvl-3d-viz ; fi
# import os
import subprocess
subprocess.Popen("cd ~/cvl-3d-viz && nohup python3 cvl/server.py > /dev/null 2>&1 &", shell=True)

<Popen: returncode: None args: 'cd ~/cvl-3d-viz && nohup python3 cvl/server....>

In [37]:
from cvl.viz import viz, VBO, Raster
visualizer = viz()
visualizer.look_at([70, 70, 5000], [1, 69.673, 0], 100)

# Input data

Data series come in tabular format and represent geographical coordinates and height. Data are hosted at Zenodo and need to be cached locally before executing the rest of the playbook. Additionally Sentinel-1 scenes are used as a background and those are added

We will pull the data zip file using requests library and unpack it locally for further use.

In [7]:
import zipfile, io, requests

url = "https://zenodo.org/record/7582591/files/MOSAiC_subset_for_CVL.zip?download=1"
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("data")

#  Load measurements as Geopandas DataFrame 

In [33]:
import pandas
df = pandas.read_csv("data/transect_latlon.txt", header=0)

In [83]:
import geopandas as gpd
# Create a geometry aware Geopandas DataFrame object 
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.Lon, df.Lat),
    crs=4326
).to_crs(32633)
print(gdf.to_crs(32633))
p0=gdf.geometry[0]

print(p0.x)

           Lon        Lat  ...  thickness                        geometry
0     0.026922  79.999070  ...       10.0  POINT (212809.298 8918661.977)
1     0.026860  79.999059  ...       10.0  POINT (212807.817 8918661.069)
2     0.026831  79.999054  ...       10.0  POINT (212807.134 8918660.722)
3     0.026780  79.999042  ...       10.0  POINT (212805.838 8918659.646)
4     0.026731  79.999031  ...       10.0  POINT (212804.618 8918658.743)
...        ...        ...  ...        ...                             ...
1080  0.027273  79.999087  ...       10.0  POINT (212816.366 8918662.043)
1081  0.027218  79.999089  ...       10.0  POINT (212815.406 8918662.615)
1082  0.027187  79.999090  ...       10.0  POINT (212814.851 8918662.858)
1083  0.026958  79.999105  ...       10.0  POINT (212810.968 8918665.564)
1084  0.026871  79.999127  ...       10.0  POINT (212809.961 8918668.406)

[1085 rows x 6 columns]
212809.29842780228


# Plot data using 3D Viz

In [44]:
# This function was taken from the 3D Viz example notebook
def post_3d_data(key, primitive, vertices, color=None, texcoord=None, normal=None, index=None, texture=None, compute_normals=False):
    #metadata = { "path" : "Examples", "time_start" : time.time()-601, "time_stop" : time.time()-601 }
    metadata = { "path" : "Examples" }
    vbo = VBO(primitive, projection=32633)
    vbo.compute_normals = compute_normals
    vbo.set_vertex(vertices)
    vbo.set_color(color)
    vbo.set_texcoord(texcoord)
    vbo.set_normal(normal)
    vbo.set_index(index)
    vbo.texture = texture
    visualizer.publish_vbo(key, metadata, vbo)

In [88]:
# Synthetic data points plotting
import numpy as np
import math

points = np.array([x for x in zip(gdf.geometry.x, gdf.geometry.y, gdf.thickness*100) ])
colors = np.ones((points.shape[0],4), dtype=np.uint32) * (125, 135, 125, 10)
vcolors = np.array([VBO.rgb_to_color(x[0], x[1], x[2], x[3]) for x in colors], dtype=np.uint32)

# Post transect data as points to web app
post_3d_data("Transect points", "points", points, vcolors)


## Screenshot from 3D Viz with the result

![title](./screenshot.png)